In [1]:
    import crewai
    from crewai import Agent, Task, Crew, Process
    from ollama import Client

In [2]:

# Initialize the Ollama client with the base URL as the 'host' argument
ollama_client = Client(host='http://localhost:11434')

In [3]:
pip install crewai requests beautifulsoup4


Note: you may need to restart the kernel to use updated packages.


In [3]:
from crewai import Agent, Task, Crew, Process
import requests
from bs4 import BeautifulSoup

# --- Helper Function: Scrape Screener.com for company data ---
def scrape_screener_data(company_name):
    query = company_name.replace(" ", "+")
    search_url = f"https://www.screener.in/search/?q={query}"

    search_resp = requests.get(search_url)
    soup = BeautifulSoup(search_resp.content, "html.parser")
    links = soup.select("a")

    # Get first valid company link
    company_link = None
    for link in links:
        href = link.get("href")
        if href and href.startswith("/company/") and not href.endswith("/"):
            company_link = "https://www.screener.in" + href
            break

    if not company_link:
        return "Company not found on Screener."

    company_resp = requests.get(company_link)
    company_soup = BeautifulSoup(company_resp.content, "html.parser")
    company_data = company_soup.get_text()
    return company_data[:3000]  # Limit to 3000 chars to avoid overload


# --- Input: Company Name ---
company_name = "Infosys"

# --- Step 1: Define Agents ---
company_analyzer = Agent(
    role="Company Analyzer",
    goal="Analyze company fundamentals and industry overview.",
    backstory="An equity analyst skilled in understanding companies and their performance.",
    verbose=True,
    llm="ollama/llama3.2:latest",
    llm_config={"provider": "ollama"}
)

screener_scraper = Agent(
    role="Financial Data Scraper",
    goal="Extract current financial metrics from Screener.in for the given company.",
    backstory="A scraping agent that extracts the most relevant public market data from Screener.",
    verbose=True,
    llm="ollama/llama3.2:latest",
    llm_config={"provider": "ollama"}
)

investment_recommender = Agent(
    role="Investment Advisor",
    goal="Based on all analysis, give a clear investment recommendation (Buy/Hold/Sell) with justification.",
    backstory="A CFA-level investment analyst specializing in stock recommendations.",
    verbose=True,
    llm="ollama/llama3.2:latest",
    llm_config={"provider": "ollama"}
)


# --- Step 2: Define Tasks ---
company_analysis_task = Task(
    description=f"""You are analyzing the company '{company_name}'. 
Research and summarize the company's business model, market position, key financial indicators, and recent news or trends.""",
    expected_output="Detailed company overview with qualitative insights.",
    agent=company_analyzer
)

scraper_task = Task(
    description=f"""Fetch and summarize the following company's financial data from Screener.in:
Company: {company_name}

Use current data available from public sources like Screener to gather:
- PE, ROE, ROCE, Debt/Equity
- Profit trends
- Key valuation ratios
- Any red flags (e.g., low promoter holding, debt, negative cash flow).""",
    expected_output="Structured summary of Screener data.",
    agent=screener_scraper
)

recommendation_task = Task(
    description="Using both the company analysis and Screener data, provide a final recommendation (Buy, Hold, or Sell) with clear reasons based on valuation, growth, and risks.",
    expected_output="Concise investment recommendation with reasoning.",
    agent=investment_recommender,
    input_tasks=[company_analysis_task, scraper_task]
)


# --- Step 3: Build Crew ---
investment_crew = Crew(
    agents=[company_analyzer, screener_scraper, investment_recommender],
    tasks=[company_analysis_task, scraper_task, recommendation_task],
    verbose=True,
    process=Process.sequential
)


# --- Step 4: Inject Screener Data via scraping ---
screener_data = scrape_screener_data(company_name)
scraper_task.description += f"\n\nScreener Data (scraped):\n{screener_data}"


# --- Step 5: Run the Crew ---
print(f"🔎 Starting Investment Analysis for: {company_name}")
result = investment_crew.kickoff()

print("\n📈 Final Recommendation:\n")
print(result)


🔎 Starting Investment Analysis for: Infosys


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0592084b-078a-4723-9877-d7aaa715cc68                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 78e94e43-90fd-4943-bb14-2d58b4430c6e
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 78e94e43-90fd-4943-bb14-2d58b4430c6e
       Status: Executing Task...
    └── 🤖 Agent: Company Analyzer
            Status: In Progress

# Agent: Company Analyzer
## Task: You are analyzing the company 'Infosys'. 
Research and summarize the company's business model, market position, key financial indicators, and recent news or trends.


...

🚀 Crew: crew
├── 📋 Task: 78e94e43-90fd-4943-bb14-2d58b4430c6e
│      Assigned to: Company Analyzer
│      Status: ✅ Completed
│   └── 🤖 Agent: Company Analyzer
│           Status: ✅ Completed
├── 📋 Task: ce5b934e-76ec-4ebc-be22-75a1a2da73ab
│      Assigned to: Financial Data Scraper
│      Status: ✅ Completed
│   └── 🤖 Agent: Financial Data Scraper
│           Status: ✅ Completed
└── 📋 Task: 89cd1e19-bff4-4219-90c3-51395fcca6e7
       Status: Executing Task...
    └── 🤖 Agent: Investment Advisor
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 78e94e43-90fd-4943-bb14-2d58b4430c6e
│      Assigned to: Company Analyzer
│      Status: ✅ Completed
│   └── 🤖 Agent: Company Analyzer
│           Status: ✅ Completed
├── 📋 Task: ce5b934e-76ec-4ebc-be22-75a1a2da73ab
│      Assigned to: Financial Data Scraper
│      Status: ✅ Completed
│   └── 🤖 Agent: Financial Data Scraper
│           Status: ✅ Completed
└── 📋 Task: 89cd1e19-bff4-4219-90c3-51395fcca6e7
       Assigned to: Investment Advisor
       Status: ✅ Completed
    └── 🤖 Agent: Investment Advisor
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 89cd1e19-bff4-4219-90c3-51395fcca6e7                                                                     │
│  Agent: Investment Advisor                                                                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 0592084b-078a-4723-9877-d7aaa715cc68                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


📈 Final Recommendation:

Based on my analysis of Infosys Limited (Infosys) as an investment opportunity, I recommend a **Buy** position with a strong conviction level of 8/10. Here's a detailed justification for this recommendation:

**Valuation Analysis:** 
At a market capitalization of $53 billion and a price-to-earnings ratio (P/E) of approximately 20, Infosys appears to be reasonably valued compared to its industry peers. The company has consistently delivered strong revenue growth, with a compound annual growth rate (CAGR) of 10% over the past five years.

**Growth Analysis:** 
Infosys has a proven track record of delivering strong growth, driven by its diversified service offerings and expanding presence in emerging markets. The company's digital transformation services segment has seen significant growth, and I expect this trend to continue as more clients adopt cloud-based solutions and embrace digital transformation.

**Risks Analysis:** 
While Infosys operates in a competiti